In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=eec8062f58124347081e47351d0df828484cdb4d381d7ad0841b5120534fb41f
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F7 '''
def F7(X):
    f = bn.F7()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.1377 ,random_state=0 , l1_ratio=0.9191, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 26.88 , C = 949.08 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_7_4000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_7_800Samples.csv').iloc[:,-1]
path = "train_latent_200D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_200D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=874421, Mon Apr 13 17:26:37 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.333199300966816e+04 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:26:37 2020)
final/bestever f-value = 2.333199e+04 2.333199e+04
incumbent solution: [0.84606631 0.92499282 0.25892754 0.75075037 0.40361693 0.4875632
 0.00798703 0.4091378  ...]
std deviations: [0.15032409 0.15019253 0.15025448 0.1503017  0.15017211 0.15028943
 0.15025892 0.15023391 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=876749, Mon Apr 13 17:26:37 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.315352062678616e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [23331.99300967] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=23331.993010, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=23331.993010, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Mon Apr 13 17:26:37 2020)
final/bestever f-value = 2.315352e+04 2.315352e+04
incumbent solution: [0.72622668 0.97925097 0.38123787 0.74863699 0.54764891 0.62772769
 0.13932673 0.46493907 ...]
std deviations: [0.15054147 0.15051706 0.15050775 0.150542   0.15057951 0.15054081
 0.15053202 0.15044611 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=912993, Mon Apr 13 17:26:37 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.675706501679288e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
    2     34 1.609643600401686e+04 1.0e+00 1.47e-01  1e-01  1e-01 0:00.1
    3     51 1.499325217615680e+04 1.0e+00 1.44e-01  1e-01  1e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [18961.50414085] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [18540.56773202] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   85   1445 -6.548627697016704e+06 1.2e+00 2.43e+00  2e+00  2e+00 0:03.1
  100   1700 -2.859533573935238e+07 1.2e+00 4.90e+00  5e+00  5e+00 0:03.7
  170   2890 -2.625916863680353e+10 1.2e+00 1.58e+02  2e+02  2e+02 0:06.2
termination on tolfacupx=1000.0 (Mon Apr 13 17:26:44 2020)
final/bestever f-value = -2.594368e+10 -2.625917e+10
incumbent solution: [-261.10467245   49.81020343  397.84979818  -14.19944976 -819.95044286
 -296.36740256 -653.46010113  199.8442388  ...]
std deviations: [155.34152896 158.33153078 154.78119899 154.10104356 156.41495432
 155.59496566 155.96129071 156.07472329 ...]
Run : 1
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=874219, Mon Apr 13 17:26:44 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.333199300966816e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:26:44 2020)
final/bestever f-value = 2.333199e+04 2.333199e+04
incumbent solution: [0.160931   0.3115157  0.621

...

   86   1462 -1.130163263304579e+07 1.4e+00 2.81e+00  3e+00  3e+00 0:03.1
  100   1700 -4.507284229133987e+07 1.4e+00 5.66e+00  6e+00  6e+00 0:03.6
  172   2924 -3.647557915230960e+10 1.4e+00 1.58e+02  2e+02  2e+02 0:06.2
termination on tolfacupx=1000.0 (Mon Apr 13 17:30:15 2020)
final/bestever f-value = -3.571852e+10 -3.647558e+10
incumbent solution: [-424.75389729  560.63055604  551.00335659 -478.21816331 -312.51863349
  231.93254243 -528.99256564  594.90561647 ...]
std deviations: [155.69720439 155.54602859 155.13132055 157.41195762 155.95856732
 155.85032851 156.1292256  156.27908475 ...]
Run : 29
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=938412, Mon Apr 13 17:30:15 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.333199300966816e+04 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:30:15 2020)
final/bestever f-value = 2.333199e+04 2.333199e+04
incumbent solution: [0.14959635 0.77327091 0.79

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [23331.99300967] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=23331.993010, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=23331.993010, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Mon Apr 13 17:30:15 2020)
final/bestever f-value = 2.315352e+04 2.315352e+04
incumbent solution: [0.08015747 0.80424388 0.84374517 1.08909994 0.21244029 0.12171259
 0.48576096 0.47923199 ...]
std deviations: [0.15072495 0.15083004 0.15058733 0.1508298  0.15072251 0.15072606
 0.1507753  0.15074245 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=951925, Mon Apr 13 17:30:15 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.187786476127584e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
    2     34 2.032113939174230e+04 1.0e+00 1.47e-01  1e-01  1e-01 0:00.1
    3     51 1.806597925808815e+04 1.0e+00 1.44e-01  1e-01  1e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [28021.01156453] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [21512.29893539] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   87   1479 -1.181854673143102e+07 1.3e+00 2.99e+00  3e+00  3e+00 0:03.1
  100   1700 -4.205687009221732e+07 1.3e+00 5.94e+00  6e+00  6e+00 0:03.6
  162   2754 -2.857200208639159e+10 1.3e+00 1.62e+02  2e+02  2e+02 0:05.8
termination on tolfacupx=1000.0 (Mon Apr 13 17:30:22 2020)
final/bestever f-value = -2.814334e+10 -2.857200e+10
incumbent solution: [ 324.96845094  -67.35768444  239.8558817   201.5399885  -867.83441547
 -346.5725641  -605.39893021  142.51475745 ...]
std deviations: [159.62790805 158.4538325  159.23704817 159.76935518 160.1275412
 158.88919122 159.3641771  159.62179952 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F7_200_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F7(X_Values.iloc[i,:100])
    SVM_Fun [i] = F7(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F7(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([2671.93847739]), count=array([1])) 146.7024257618907
SVM
ModeResult(mode=array([2733.25969742]), count=array([1])) 192.22175776519452
ELN
ModeResult(mode=array([1.53879976e+08]), count=array([1])) 24717051.148459245
